In [1]:
import requests as req
from bs4 import BeautifulSoup as bs

In [2]:
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 11.8 MB/s eta 0:00:00


In [3]:
#링크가져와서 요청 후 서버에게 응답받기
res = req.get("https://www.vogue.co.kr/fashion/fashion-trend/")
res

<Response [200]>

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_article_text_selenium(url):
    if not url: return ""

    # 크롬 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless") # 창 없이 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    # 드라이버 자동 설치 및 실행
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)

        # 본문이 로드될 때까지 최대 10초 대기 (중요!)
        # 보그의 본문 클래스 중 하나가 나타날 때까지 기다립니다.
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.post-content, section.post-content, .entry-content')))

        # 페이지 소스를 가져와서 다시 BeautifulSoup으로 분석 (속도 향상)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 보그 본문 타겟팅
        content = soup.select_one('div.post-content') or \
                  soup.select_one('section.post-content') or \
                  soup.select_one('.entry-content')

        if content:
            # 광고 등 불필요 요소 제거
            for bad in content.select("script, style, .sharedaddy, .wp-caption-text"):
                bad.decompose()
            return content.get_text(" ", strip=True)
        else:
            return "본문을 찾을 수 없습니다."

    except Exception as e:
        return f"에러 발생: {e}"
    finally:
        driver.quit() # 브라우저 반드시 종료

In [5]:
!pip install lxml


In [6]:
from tqdm.auto import tqdm # 진행 상황 확인용
import time

print("--- 셀레니움을 이용한 본문 추출 시작 (시간이 다소 소요됩니다) ---")

# posts 리스트에 있는 각 기사들을 순회하며 본문 수집
for a in tqdm(posts, desc="기사 수집 중"):
    # 이미 본문이 채워져 있지 않은 경우에만 실행
    if "content" not in a or "본문을 찾을 수 없습니다" in a["content"] or not a["content"]:
        # 위에서 정의한 셀레니움 함수 호출
        a["content"] = get_article_text_selenium(a["link"])
        # 서버 부하를 줄이기 위해 짧은 휴식 (선택 사항)
        time.sleep(1)

print("--- 모든 기사 본문 수집 완료! ---")

# 결과 확인 (첫 번째 기사)
if posts:
    print(f"\n[수집 확인 - {posts[0]['title']}]")
    print(f"본문 길이: {len(posts[0].get('content', ''))}자")
    print(f"내용 요약: {posts[0].get('content', '')[:150]}...")

--- 셀레니움을 이용한 본문 추출 시작 (시간이 다소 소요됩니다) ---


NameError: name 'posts' is not defined

In [ ]:
#html형식으로 파싱
soup = bs(res.text, 'lxml')
soup

In [ ]:
!pip install pandas
!pip install konlpy

In [ ]:
!pip install kiwipiepy

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def get_article_text_selenium_final(url):
    if not url: return ""

    chrome_options = Options()
    chrome_options.add_argument("--headless=new") # 최신 헤드리스 모드 사용
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    # 봇 감지 방지 추가
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    try:
        # 드라이버 설치 시 로그 출력을 줄여서 오류 추적을 쉽게 함
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        driver.get(url)
        # 페이지가 안정화될 때까지 대기
        time.sleep(4)

        # 스크롤을 살짝 내려서 동적 콘텐츠 활성화 (보그 같은 사이트에 효과적)
        driver.execute_script("window.scrollTo(0, 500);")
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 보그 코리아의 다양한 본문 클래스 시도
        content = soup.select_one('div.post-content') or \
                  soup.select_one('div.entry-content') or \
                  soup.select_one('section.post-content')

        if content:
            # 텍스트 추출 후 깔끔하게 정리
            res_text = content.get_text(" ", strip=True)
            if len(res_text) > 100:
                return res_text

        # 최후의 수단: 특정 길이 이상의 p 태그 다 모으기
        p_text = " ".join([p.get_text(strip=True) for p in soup.select('p') if len(p.get_text()) > 30])
        return p_text if len(p_text) > 50 else "본문 추출 실패"

    except Exception as e:
        return f"에러 발생: {str(e)}"
    finally:
        try:
            driver.quit()
        except:
            pass

# 딱 한 개만 다시 테스트 해보세요!
test_res = get_article_text_selenium_final(posts[0]['link'])
print(f"테스트 결과: {test_res[:200]}...")

In [ ]:
from tqdm.auto import tqdm
import time

print("--- 🚀 전체 기사 본문 수집 시작 (성공한 코드로 재실행) ---")

for a in tqdm(posts):
    # 기존에 에러 메시지가 들어있거나 비어있는 경우에만 수집
    if "에러 발생" in a.get('content', '') or "본문 추출 실패" in a.get('content', '') or not a.get('content'):
        a["content"] = get_article_text_selenium_final(a["link"])
        time.sleep(1) # 서버 보호를 위한 잠깐의 휴식

print("--- ✅ 모든 기사 수집 완료! ---")

In [ ]:
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()

# 1. 수집된 모든 본문 텍스트 합치기
all_content = " ".join([a.get('content', '') for a in posts if a.get('content')])

# 2. 명사 추출 (NNG: 일반 명사, NNP: 고유 명사)
# 2글자 이상의 단어만 필터링합니다.
tokens = kiwi.tokenize(all_content)
words = [t.form for t in tokens if t.tag in ['NNG', 'NNP'] and len(t.form) > 1]

# 3. 빈도수 계산 및 상위 30개 출력
count = Counter(words)
most_common = count.most_common(30)

print("--- 🏆 2026 패션 트렌드 분석 결과 (Top 30) ---")
for i, (word, freq) in enumerate(most_common, 1):
    print(f"{i:2d}위. {word}: {freq}회")

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 워드클라우드 생성 (상위 100개 단어 대상)
# 윈도우 폰트 경로 (맥 사용 시 /Library/Fonts/AppleGothic.ttf 로 변경)
font_path = "c:/Windows/Fonts/malgun.ttf"

wc = WordCloud(
    font_path=font_path,
    background_color="white",
    width=1000,
    height=700,
    max_words=100,
    colormap='Set2' # 색상 테마
)

# 빈도수 데이터를 바탕으로 생성
cloud = wc.generate_from_frequencies(dict(count.most_common(100)))

# 화면 출력
plt.figure(figsize=(12, 9))
plt.imshow(cloud, interpolation='bilinear')
plt.axis('off') # 축 숨기기
plt.title("2026 Vogue Fashion Trend WordCloud", size=20)
plt.show()

In [ ]:
import pandas as pd

# 열 너비 제한을 완전히 해제 (None으로 설정)
pd.set_option('display.max_colwidth', None)

# 다시 출력해보기
df_final[['title', 'link']]

In [ ]:
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()

# 기사별로 키워드를 뽑는 함수 정의
def get_top_nouns(text, top_n=5):
    if not text or len(text) < 10:
        return "내용 없음"

    # 명사 추출
    tokens = kiwi.tokenize(text)
    words = [t.form for t in tokens if t.tag in ['NNG', 'NNP'] and len(t.form) > 1]

    # 빈도수 계산 후 상위 N개 추출
    counts = Counter(words)
    top_words = counts.most_common(top_n)

    # 보기 좋게 문자열로 결합 (예: 패션(10), 트렌드(8)...)
    return ", ".join([f"{word}({cnt})" for word, cnt in top_words])

# 데이터프레임의 'content' 열에 함수 적용
df_final['top_keywords'] = df_final['content'].apply(lambda x: get_top_nouns(x))

# 결과 확인 (제목, 링크, 그리고 추출된 키워드)
df_final[['title', 'link', 'top_keywords']]

In [ ]:
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()

# 1. 제거하고 싶은 단어들 추가 (불용어 리스트)
stopwords = ['하솔휘', '에디터', '이번', '통해', '생각', '우리', '모습']

# 0번 인덱스(핀터레스트 기사) 분석
target_index = 0
content_sample = df_final.loc[target_index, 'content']
title_sample = df_final.loc[target_index, 'title']

tokens = kiwi.tokenize(content_sample)

# 2. 명사 추출 시 불용어 리스트에 없는 단어만 필터링
words = [
    t.form for t in tokens
    if t.tag in ['NNG', 'NNP']             # 일반/고유 명사만
    and len(t.form) > 1                    # 1글자 제외
    and t.form not in stopwords            # ⭐ 불용어 제외
]

# 3. 빈도수 계산
detail_count = Counter(words).most_common(20)

print(f"[{title_sample}] 상세 키워드 빈도수 (불용어 제거 후)")
for i, (word, freq) in enumerate(detail_count, 1):
    print(f"{i:2d}위: {word} ({freq}회)")

In [ ]:
from kiwipiepy import Kiwi
from collections import Counter

kiwi = Kiwi()

# 1. 분석할 대상 URL
target_url = "https://www.vogue.co.kr/2025/12/15/2026%eb%85%84%ea%b9%8c%ec%a7%80-%eb%91%90%ea%b3%a0%eb%91%90%ea%b3%a0-%ed%99%9c%ec%9a%a9%ed%95%a0-%ec%b2%ad%eb%b0%94%ec%a7%80-%ec%8a%a4%ed%83%80%ec%9d%bc%eb%a7%81-8/"

# 2. 데이터프레임에서 해당 기사 본문 가져오기
target_row = df_final[df_final['link'] == target_url]

if not target_row.empty:
    content = target_row.iloc[0]['content']
    title = target_row.iloc[0]['title']

    # 3. 불용어 설정 (에디터 이름 등 추가)
    stopwords = ['하솔휘', '에디터', '이번', '통해', '가지', '스타일링', '생각', '우리','웨이','방식','배럴']

    # 4. 명사 추출 및 필터링
    tokens = kiwi.tokenize(content)
    words = [t.form for t in tokens if t.tag in ['NNG', 'NNP'] and len(t.form) > 1 and t.form not in stopwords]

    # 5. 빈도수 계산
    count_result = Counter(words).most_common(20)

    print(f"[{title}] 핵심 키워드 Top 20")
    print("-" * 40)
    for i, (word, freq) in enumerate(count_result, 1):
        print(f"{i:2d}위: {word} ({freq}회)")
else:
    print("해당 링크의 기사를 데이터프레임에서 찾을 수 없습니다. 수집 여부를 확인해주세요.")

In [ ]:
import pandas as pd

# 1. 수집한 리스트(posts)를 데이터프레임으로 변환
df_final = pd.DataFrame(posts)

# 2. 파일로 저장
# encoding='utf-8-sig'를 해야 엑셀에서 한글이 깨지지 않고 잘 보입니다.
file_name = "vogue_2026_fashion_data.csv"
df_final.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"✅ 저장 완료! 파일명: {file_name}")
print(f"저장된 기사 개수: {len(df_final)}개")

In [ ]:
# 1. 패션 맥락에 맞는 감성 단어 정의
pos_words = ['혁신', '우아', '세련', '아름답', '완벽', '매력', '주목', '화려', '새롭', '돋보이', '트렌디', '필수', '감각']
neg_words = ['지루', '낡', '한계', '사라지', '물러나', '어렵', '단조롭', '불안', '과하', '복잡', '불편']

def get_sentiment_score(text):
    if not text: return 0
    tokens = kiwi.tokenize(text)
    score = 0
    for t in tokens:
        if t.form in pos_words: score += 1
        if t.form in neg_words: score -= 1
    return score

# 2. 모든 기사에 적용
df_final['sentiment_score'] = df_final['content'].apply(get_sentiment_score)

# 3. 긍정 점수가 높은 순으로 정렬하여 트렌드 순위 확인
trend_ranking = df_final[['title', 'sentiment_score']].sort_values(by='sentiment_score', ascending=False)

print("--- 🌟 가장 긍정적으로 묘사된 트렌드 순위 ---")
print(trend_ranking)

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm.auto import tqdm

# 1. 인증이 필요 없는 안정적인 한국어 감성 분석 모델로 변경
# jaehyeong/koelectra-base-v3-generalized-sentiment-analysis 모델 사용
try:
    sentiment_model = pipeline(
        "sentiment-analysis",
        model="jaehyeong/koelectra-base-v3-generalized-sentiment-analysis",
        device=-1  # GPU 사용 시 0으로 변경
    )
    print("✅ 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")

def get_deep_sentiment(text):
    if not text or len(text) < 10:
        return 0.5

    # 모델 입력 글자 수 제한(512자)을 위해 본문 앞부분 추출
    content = text[:510]
    try:
        result = sentiment_model(content)[0]
        # 모델의 출력 방식에 따라 점수 변환 (1: 긍정, 0: 부정 가정)
        # 해당 모델은 보통 '1'이 긍정, '0'이 부정으로 출력됩니다.
        label = result['label']
        score = result['score']

        # 긍정 라벨인 경우 점수 그대로, 부정이면 1에서 뺌
        if '1' in label or 'POS' in label.upper():
            return score
        else:
            return 1 - score
    except:
        return 0.5

# 2. 분석 실행
tqdm.pandas()
df_final['deep_sentiment_score'] = df_final['content'].progress_apply(get_deep_sentiment)

# 3. 결과 확인
deep_ranking = df_final[['title', 'deep_sentiment_score']].sort_values(by='deep_sentiment_score', ascending=False)
print("\n--- 🤖 AI 기반 정교한 트렌드 긍정도 순위 (수정본) ---")
for i, (idx, row) in enumerate(deep_ranking.iterrows(), 1):
    status = "🌟 매우 긍정" if row['deep_sentiment_score'] > 0.8 else "✅ 긍정" if row['deep_sentiment_score'] > 0.5 else "❓ 변화/중립"
    print(f"{i}위. [{row['title']}] - 점수: {row['deep_sentiment_score']:.2f} ({status})")

## Slide 1. [변화] "편안함에서 화려함으로"
2025 (과거): 오버사이즈 코트, 조거팬츠 (미니멀 & 편안함)

2026 (미래): 맥시멀리즘의 부활 (레이스, 자수, 정교한 디테일)

Key: 핀터레스트가 예측한 재킷과 카키 컬러의 급부상

Slide 2. [소재/컬러] "익숙한 소재의 낯선 변주"
Color: 베이지·뉴트럴 → **'카키(Khaki)' & '딥 블랙'**으로의 세대교체

Material: 울·캐시미어의 고급스러움에 레이스·브로치를 섞는 믹스매치

ESG: 전통 가죽·벨벳을 대체하는 친환경 & 테크니컬 소재 확대

Slide 3. [아이템] "기본템을 살리는 한 끗 디테일"
Denim: 평범한 청바지 대신 '2026년식 뉴 스타일링' 적용

Accessory: 바라클라바 대신 볼륨감 있는 주얼리와 브로치가 핵심

🏆 최종 결론 (One-Liner)
"정제된 럭셔리 위에 '레이스와 카키'라는 과감한 디테일을 더한 '정교한 맥시멀리즘'의 시대"

## 최종 분석 리포트: 2026 패션 트렌드 전략
1. 압도적 주류 트렌드: 애슬레저와 실용주의 (Top 1~3)
핵심 아이템: 브라 톱, 레깅스, 스키 웨어

인사이트: 2026년에도 패션의 중심은 '건강한 신체'와 '활동성'에 있습니다. 특히 브라 톱이 레깅스의 자리를 대체하거나 보완하며 더욱 과감한 애슬레저 룩이 긍정적으로 수용되고 있습니다.

2. 지속 가능한 클래식: 데님과 스테디셀러 (Top 4~6)
핵심 아이템: 신발(슈즈), 청바지, 핀터레스트 기반 예측 아이템

인사이트: 2026년까지 지속될 '청바지'와 '올해 신었던 신발'에 대한 높은 긍정도는 소비자들이 한 번 구매해서 오래 입을 수 있는(Timeless) 아이템에 강한 신뢰를 보내고 있음을 보여줍니다.

3. 새로운 스타일 공식: 패러독스 드레싱 & 컬러 (Top 7~10)
핵심 키워드: 패러독스 드레싱, 흰 티셔츠, 클라우드 댄서(화이트)

인사이트: 상반된 아이템을 섞어 입는 '패러독스 드레싱'과 팬톤이 선정한 화이트 계열 컬러가 새로운 패션 문법으로 자리 잡으며 매우 긍정적인 반응을 얻고 있습니다.